In [142]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn

### Post process the data

In [143]:
df = pd.read_csv('processedData.csv')
df.loc[:,'dx'] = df['x'].diff().fillna(0.0) 
df.loc[:,'dy'] = df['y'].diff().fillna(0.0)
df['accX'] = df.accX.ffill().add(df.accX.bfill()).div(2)
df['accY'] = df.accY.ffill().add(df.accY.bfill()).div(2)
df['accZ'] = df.accZ.ffill().add(df.accZ.bfill()).div(2)
df['gyrX'] = df.gyrX.ffill().add(df.gyrX.bfill()).div(2)
df['gyrY'] = df.gyrY.ffill().add(df.gyrY.bfill()).div(2)
df['gyrZ'] = df.gyrZ.ffill().add(df.gyrZ.bfill()).div(2) 
train, test = train_test_split(df, test_size=0.2)
train.head()

,Unnamed: 0,Unnamed: 0.1,time,accX,accY,accZ,gyrX,gyrY,gyrZ,y,x,dx,dy
122,124,124,2.456207,0.19,-0.23,9.89,0.004363,0.049087,-0.011999,-0.537081,0.047527,0.000215,-0.004243
170,173,173,3.421085,0.44,0.33,9.43,-0.058905,0.044724,-0.017453,-0.795094,0.066703,0.000584,-0.005115
63,65,65,1.298898,0.29,-0.09,9.37,0.008727,-0.051269,-0.009817,-0.247515,0.029089,0.000498,-0.004688
72,74,74,1.478649,-0.47,0.18,9.54,0.017453,-0.103629,0.041452,-0.290819,0.033169,0.000469,-0.005481
107,109,109,2.157535,0.18,-0.08,9.85,0.053451,-0.032725,0.022907,-0.461174,0.043757,0.000216,-0.004320


In [144]:
Xtrain = np.column_stack([train["accX"], train["accY"], train["accZ"], train["gyrX"], train["gyrY"], train["gyrZ"]])#.T
Ytrain = np.column_stack([train["dx"], train["dy"]])#.T
Xtest = np.column_stack([test["accX"], test["accY"], test["accZ"], test["gyrX"], test["gyrY"], test["gyrZ"]])#.T
Ytest = np.column_stack([test["dx"], test["dy"]])#.T

In [145]:
nX = np.shape(Xtrain)[1]
nH1 = 100
nH2 = 16
nY = np.shape(Ytrain)[1]
nX

6

In [146]:
class NN(nn.Module):
    
    def __init__(self, input_size, hidden1_size, hidden2_size, num_classes):
        
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden1_size)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden1_size, hidden2_size)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden2_size, num_classes)
        #self.fc3 = nn.Linear(hidden1_size, num_classes)  
           
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        return out

In [147]:
model = NN(nX, nH1, nH2, nY)
model = model.float()

### Convert data into a torch dataset

In [148]:
tensor_Xtrain = torch.from_numpy(Xtrain)
tensor_Ytrain = torch.from_numpy(Ytrain)
tensor_Xtest = torch.from_numpy(Xtest)
tensor_Ytest = torch.from_numpy(Ytest)
train_dataset = torch.utils.data.TensorDataset(tensor_Xtrain, tensor_Ytrain)
test_dataset = torch.utils.data.TensorDataset(tensor_Xtest, tensor_Ytest)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=5, shuffle=True)
# for data in train_dataset:
#     print(data)
#test_loader  = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

### Cost function and optimizer

In [149]:
cost = nn.MSELoss()
alpha = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=alpha, nesterov=True, momentum=0.9, dampening=0)  

### Train

In [150]:
episodes = 1000

for epis in range(episodes):
    for data in train_dataset:
        X, y = data
        model.zero_grad()
        output = model(X.float())#.view(-1, 6))
        loss = cost(output, y.float())
        loss.backward()
        optimizer.step()
    print(loss)

tensor(3.6416e-06, grad_fn=<MseLossBackward>)
tensor(5.0909e-06, grad_fn=<MseLossBackward>)
tensor(5.7976e-06, grad_fn=<MseLossBackward>)
tensor(6.2254e-06, grad_fn=<MseLossBackward>)
tensor(6.5788e-06, grad_fn=<MseLossBackward>)
tensor(6.8448e-06, grad_fn=<MseLossBackward>)
tensor(6.9905e-06, grad_fn=<MseLossBackward>)
tensor(7.0337e-06, grad_fn=<MseLossBackward>)
tensor(7.0082e-06, grad_fn=<MseLossBackward>)
tensor(6.9151e-06, grad_fn=<MseLossBackward>)
tensor(6.7811e-06, grad_fn=<MseLossBackward>)
tensor(6.6133e-06, grad_fn=<MseLossBackward>)
tensor(6.4230e-06, grad_fn=<MseLossBackward>)
tensor(6.2126e-06, grad_fn=<MseLossBackward>)
tensor(5.9871e-06, grad_fn=<MseLossBackward>)
tensor(5.7558e-06, grad_fn=<MseLossBackward>)
tensor(5.5231e-06, grad_fn=<MseLossBackward>)
tensor(5.2924e-06, grad_fn=<MseLossBackward>)
tensor(5.0651e-06, grad_fn=<MseLossBackward>)
tensor(4.8473e-06, grad_fn=<MseLossBackward>)
tensor(4.6364e-06, grad_fn=<MseLossBackward>)
tensor(4.4331e-06, grad_fn=<MseLos